In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
pd.set_option('display.max_columns', None)

# Procesado de los datos

## Lectura de datos

In [ ]:
# Cargamos los datos y eliminamos las filas duplicadas. Un mismo vuelo no puede retrasarse en el mismo momento del tiempo más de una vez
df = pd.read_csv("../Data/flights.csv")
df = df.drop_duplicates()
df.head()

## Arreglo de datos

In [ ]:
df['ORIGIN_AIRPORT'] = df['ORIGIN_AIRPORT'].astype(str)
df['DESTINATION_AIRPORT'] = df['DESTINATION_AIRPORT'].astype(str)

Hay una serie de códigos de aeropuertos de origen y destino que no corresponden con el IATA_CODE asociado al aeropuerto si no que aparece un id numérico.

In [ ]:
print("Aeropuertos origen: "+ str(len(df[df['ORIGIN_AIRPORT'].str.isdigit()])))
print("Aeropuertos destino: "+ str(len(df[df['DESTINATION_AIRPORT'].str.isdigit()])))

Para corregir esto hacemos uso del archivo airports_dict, el cual genera un diccionario en el que se asocian las claves de aeropuerto numéricas a un código str como el que tenemos en el resto de casos en función de las rutas de los vuelos.

In [ ]:
# Cargamos el diccionario
with open('dict_airport.json', 'rb') as fp:
    dict_airport = pickle.load(fp)

In [ ]:
# Creamos dos columnas auxiliares para reemplazar el código numérico por el codigo str de aeropuerto correspondiente
df['CODE_ORI'] = df['ORIGIN_AIRPORT']
df['CODE_ORI'] = df['CODE_ORI'].map(dict_airport)

df['CODE_DEST'] = df['ORIGIN_AIRPORT']
df['CODE_DEST'] = df['DESTINATION_AIRPORT'].map(dict_airport)

# Sustituimos
df['CODE_ORI'] = df['CODE_ORI'].fillna(df['ORIGIN_AIRPORT'])
df['ORIGIN_AIRPORT'] = df['CODE_ORI']

df['CODE_DEST'] = df['CODE_DEST'].fillna(df['DESTINATION_AIRPORT'])
df['DESTINATION_AIRPORT'] = df['CODE_DEST']

df = df.drop(['CODE_ORI','CODE_DEST'],axis = 1)

In [ ]:
# Comprobamos
print("Aeropuertos origen: "+ str(len(df[df['ORIGIN_AIRPORT'].str.isdigit()])))
print("Aeropuertos destino: "+ str(len(df[df['DESTINATION_AIRPORT'].str.isdigit()])))

**NOTA:** vemos que hay una serie de índices que el algoritmos no consigue emparejar con un aeropuerto. No obstante, estos datos no corresponden con una muestra representativa de los datos (poco volumen) por lo que los eliminamos

In [ ]:
df = df.drop(df[df['ORIGIN_AIRPORT'].str.isdigit()].index)
df = df.drop(df[df['DESTINATION_AIRPORT'].str.isdigit()].index)

In [ ]:
# Comprobamos parte II
print("Aeropuertos origen: "+ str(len(df[df['ORIGIN_AIRPORT'].str.isdigit()])))
print("Aeropuertos destino: "+ str(len(df[df['DESTINATION_AIRPORT'].str.isdigit()])))

## Análisis de variables

In [ ]:
df.columns

In [ ]:
df.info()

Columnas que se pueden eliminar:
- TAIL NUMBER: representa un ID de avión único por lo que no corresponde con una variable representativa para el análisis por lo que la eliminamos

### Debatir:
Nos importan..?
- SCHEDULED_DEPARTURE: yo diría que no, si llega en hora = OK!
- DEPARTURE_TIME: es interesante analizar los retrasos por tramo horario? - SI
- DEPARTURE_DELAY: yo diría que no, si sale tarde pero llega en hora = OK! - SI
- ARRIVAL_TIME: yo diría que no, solo me interesa si el vuelo llega tarde, no? - SI
- SCHEDULED_TIME: no se a que se refiere  
- SCHEDULED_ARRIVAL: solo nos interesa saber si se retrasa o no, no la previsión
- TAXI_IN
- TAXI_OUT
- WHEELS_OFF
- WHEELS_ON
- AIR_TIME
- FLIGHT_NUMBER - SI
- DESTINATION_AIRPORT - DUDA

In [ ]:
# Eliminamos tail number porque es un identificador de avión (no vuelo) único
df = df.drop("TAIL_NUMBER",axis=1)

In [ ]:
# Eliminamos el resto de variables que no usaremos para nuestro análisis
df = df.drop(["SCHEDULED_DEPARTURE", "SCHEDULED_TIME", "SCHEDULED_ARRIVAL",
             "TAXI_IN", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "AIR_TIME"],axis=1)

Hay un error de formato con la variable flight number, que es un indicador del vuelo y por lo tanto una variable categórica. Lo mismo ocurre con Cancelled y Diverted

In [ ]:
# Añadimos el FlightNum, Cancelled and Diverted como variables categóricas
df['FLIGHT_NUMBER']=df['FLIGHT_NUMBER'].astype(object) 
df['CANCELLED']=df['CANCELLED'].astype(object) 
df['DIVERTED']=df['DIVERTED'].astype(object) 

In [ ]:
# Unimos las columnas Year, Month y Day of Month como una única variable fecha
# Formato por defecto mes/dia/año para que to_datetime funcione correctamente
df["DATE"]  = df['MONTH'].astype(str) +'/'+ df['DAY'].astype(str) +'/' + df['YEAR'].astype(str)
df["DATE"] = pd.to_datetime(df["DATE"])

Si nos interesa en algún momento podríamos añadir las horas también 

In [ ]:
#Eliminamos las columnas year, month y day
df = df.drop(["YEAR","MONTH", "DAY"],axis=1)

# También podemos eliminar la columna DAY_OF_WEEK ya que podemos obtenerla
df = df.drop("DAY_OF_WEEK",axis=1)
print(f"Ejemplo: obtener dia de la semana de la fecha {df['DATE'].iloc[0]} --> {df['DATE'].iloc[0].day_name()} (dia {df['DATE'].iloc[0].dayofweek})")

# NOTA: hay que tener en cuenta que dayofweek empieza a contar en 0 = lunes

In [ ]:
# Cambiamos el orden de las columnas, para que DATE sea la primera
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols] 
df.head()

Hablar formato tiempos: 
    SCHEDULED_DEPARTURE, DEPARTURE_TIME, SCHEDULED_TIME
    SCHEDULED_ARRIVAL, ARRIVAL_TIME (se podrían eliminar dado que nos sirven para sacar arrival delay)

## Debatir:

Las variables SCHEDULED_DEPARTURE, DEPARTURE_TIME, DEPARTURE_DELAY, ARRIVAL_TIME y SCHEDULED_ARRIVAL nos ayudan a saber si un vuelo se ha retrasado o no, pero una vez aue sabemos esto no tienen  valor en sí mismas por lo que las podemos eliminar

#### Tipos de vuelo:
1. On time/ arrived earlier --> arrival_delay <=0
2. Delayed   --> arrival_delay > 0
3. Diverted  --> diverted == 1
4. Cancelled --> cancelled == 1

##### Razones por las que se retrasa un vuelo:
- AIR_SYSTEM_DELAY     
- SECURITY_DELAY       
- AIRLINE_DELAY        
- LATE_AIRCRAFT_DELAY 
- WEATHER_DELAY       

In [ ]:
# Vamos a examinar ahora los valores nulos
df.isna().sum()

Las variables CANCELLATION_REASON, AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY,WEATHER_DELAY presentan una gran cantidad de valores nulos. No obstante, todas estas variables se relacionan con vuelos cancelados o retrasados por lo que tiene sentido que sean valores nulos para aquellos vuelos que no se hayan cancelado ni retrasado. Vamos a analizar estas variables

In [ ]:
cancelled = df[df['CANCELLED'] == 1]
cancelled.isna().sum()

Vemos como en este caso no existen valores nulos para la columna CANCELLATION_REASON. No obstante, nuestro análisis consiste en prededcir el retraso de vuelos por lo que no necesitamos la info de vuelos cancelados/ redirigidos así que eliminamos dichos registros

In [ ]:
# Eliminamos los cancelados
df = df[df['CANCELLED'] == 0]

#Eliminamos los redirigidos
df = df[df['DIVERTED'] == 0]

df = df.drop(["DIVERTED","CANCELLED", "CANCELLATION_REASON"],axis=1)

Nos quedamos solo con los vuelos que llegan antes de lo previsto, en hora o con retraso

In [ ]:
len(df)

In [ ]:
df.isna().sum()

Realizamos ahora el análisis de los valores nulos relacionados con los vuelos retrasados.

**NOTA**: consideramos que un vuelo se retrasa si llega pasada la hora prevista, independientemente de si ha tenido retraso en la hora de salida o no

In [ ]:
delayed = df[df['ARRIVAL_DELAY'] > 0]
delayed.isna().sum()

Si considerabamos todos los vuelos teníamos 1283118 registros con valor nulo en las variables DELAY, cuando filtramos por vuelos retrasados tenemos 275025.
Vamos a analizar primero los valores nulos de aquellos vuelos que se han retrasado

In [ ]:
delayed = delayed[['AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']]
delayed.head()

In [ ]:
# Seleccionamos todas las filas que tengan NaN en todas las columnas
nulls = delayed.loc[(delayed['AIR_SYSTEM_DELAY'].isnull() == True) & (delayed['SECURITY_DELAY'].isnull() == True) & (delayed['AIRLINE_DELAY'].isnull() == True) & (delayed['LATE_AIRCRAFT_DELAY'].isnull() == True) & (delayed['WEATHER_DELAY'].isnull() == True)]
len(nulls)

Vemos que TODOS los NaN se concentran en las mismas filas. Entendemos que en este caso el vuelo se ha retrasado por causa desconocida. Para indicar esto creamos una nueva columna 'OTHER_DELAY' en nuestro data frame cuyo valor sea igual al delay

In [ ]:
# Creamos la OTHER_DELAY con los mismos datos que ARRIVAL_DELAY
df['OTHER_DELAY'] = df['ARRIVAL_DELAY']

# Como hemos visto que si una columna _DELAY es NaN el resto también, utilizamos una única columa para comparar, y asignamos a OTHER_DELAY la diferencia entre el delay a la llegada y el resto de delays
df.loc[pd.notna(df['AIR_SYSTEM_DELAY']),'OTHER_DELAY'] = df['ARRIVAL_DELAY'] - df['AIR_SYSTEM_DELAY'] - df['SECURITY_DELAY'] - df['AIRLINE_DELAY'] -df['LATE_AIRCRAFT_DELAY'] - df['WEATHER_DELAY']
df[45:50]

In [ ]:
# Cambiamos OTHEY_DELAY <0 por 0 dado que estos vuelos han llegado antes de lo previsto, no han experimentado un retraso
df.loc[df["OTHER_DELAY"] < 0, "OTHER_DELAY"] = 0
df[45:50]

In [ ]:
# Early arrival flights
early_arrival = df[df['ARRIVAL_DELAY']<=0]

print(len(early_arrival))
early_arrival.isna().sum()

El resto de valores NaN en dichas columnas corresponden a aquellos vuelos que han llegado antes de tiempo a destino, cosa que tiene sentido dado que no han experimentado ningún delay

Una vez analizado el por qué de los valores NaN presentes en el dataset parece razonable sustituir dichos valores por 0

In [ ]:
# Asignamos a todos los NaNs el valor 0, ya que ahora todo el retraso de sus vuelos está plasmado en la variable OTHER_DELAY
df = df.fillna(0)
df[45:50]

fillna() sustituye los NaN de todo el dataframe por lo que es importante destacar que podemos utilizar este método dado que los únicos campos NaN del dataframe se encuentran en las variables de delay analizadas. 

In [ ]:
df.isna().sum()

Resulta interesante saber si los vuelos pueden retrasarse por un único motivo o exclusivamente por uno. Vamos a investigarlo

In [ ]:
# Tomamos la variable WEATHER_DELAY como referencia
weather = df.loc[(df['WEATHER_DELAY'] != df['ARRIVAL_DELAY']) & (df['WEATHER_DELAY']>0 )]
weather.head()

Confirmamos, el retraso puedes estar asociado a varios motivos

## CHECKPOINT. Datos de vuelos limpios

In [ ]:
df.head()

## Info Aerolíneas

In [ ]:
# Vamos a añadir la el nombre asociado a las airlines
# Cargamos los datos
airlines = pd.read_csv("../Data/airlines.csv")
airlines.head()

In [ ]:
# Vamos a examinar ahora los valores nulos
airlines.isna().sum()

In [ ]:
# Renombramos la columna "AIRLINE" para poder hacer el join con la tabla de aerolineas
airlines = airlines.rename(columns={"IATA_CODE": "AIRLINE_CODE"})
df = df.rename(columns={"AIRLINE": "AIRLINE_CODE"})
df.head()

In [ ]:
# Unimos ambas tablas
flights = df.merge(airlines, on='AIRLINE_CODE', how='left')

In [ ]:
# Ponemos la nueva columna a continuación del código de la aerolinea
cols = flights.columns.tolist()
cols = cols[0:2]+cols[-1:] + cols[2:-1]
flights = flights[cols] 
flights.head()

## Info Aeropuertos

In [ ]:
# Vamos a añadir la el nombre asociado a las airlines
# Cargamos los datos
airports = pd.read_csv("../Data/airports.csv")
airports.head()

In [ ]:
# Vamos a examinar ahora los valores nulos
airports.isna().sum()

Falta información de 3 aeropuertos, así que la rellenaremos buscando sus datos en internet e introduciéndola manualmente

In [ ]:
ECP_COORD = [30.3549, 85.7995]
PBG_COORD = [44.6521, 73.4679]
UST_COORD = [29.9544, 81.3429]
airports.at[96,["LATITUDE","LONGITUDE"]]= ECP_COORD
airports.at[234,["LATITUDE","LONGITUDE"]]= PBG_COORD
airports.at[313,["LATITUDE","LONGITUDE"]]= UST_COORD
airports.isna().sum()

Eliminamos la columna COUNTRY, ya que todos los aerupuertos son de Estados Unidos

In [ ]:
airports = airports.drop("COUNTRY", axis=1)

Cambiamos el nombre de la columna airport, ya que trabajaremos únicamente con los códigos de los aeropuertos y con sus nombres

In [ ]:
airports= airports.rename(columns={"AIRPORT": "AIRPORT_NAME"})
airports= airports.rename(columns={"IATA_CODE": "AIRPORT"})

In [ ]:
# Creamos dos bases de datos para hacer el join con la principal tanto para aeropuertos de llegada como de salida
airports= airports.rename(columns={"IATA_CODE": "AIRPORT"})

origin_airports = airports.add_prefix('ORIGIN_')
destination_airports = airports.add_prefix('DESTINATION_')

origin_airports.head()

In [ ]:
# Unimos las tres tablas
flights_origin = flights.merge(origin_airports, on='ORIGIN_AIRPORT', how='left')
flights_complete = flights_origin.merge(destination_airports, on='DESTINATION_AIRPORT', how='left')
flights_complete.head()

In [ ]:
len(flights_complete)

In [ ]:
# Guardamos los datos preprocesados, para ser utilizados en la predicción posterior
flights_complete.to_parquet("flightsCleaned.parquet", index=False)